In [1]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import json

In [35]:
# Step 1: Load the dataset
def load_data(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    # Combine prompt and response for causal language modeling
    return [{"text": f"<|startoftext|>{item['prompt']} {item['response']}<|endoftext|>"} for item in data]

data = load_data("chat_data.json")
dataset = Dataset.from_list(data)

In [36]:
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 50
})


In [37]:
# Step 2: Tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def preprocess(data):
    inputs = tokenizer(
        data["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )
    # Set the labels to be the same as input_ids for causal LM
    inputs["labels"] = inputs["input_ids"].copy()
    return inputs

tokenized_data = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [49]:
# Step 3: Load the model
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Step 4: Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50,
    evaluation_strategy="no",
    report_to="none",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [50]:
# Step 5: Fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    tokenizer=tokenizer,
)

trainer.train()

<ipython-input-50-3fae56c5a340>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,0.945400


TrainOutput(global_step=75, training_loss=0.7338128089904785, metrics={'train_runtime': 22.7893, 'train_samples_per_second': 6.582, 'train_steps_per_second': 3.291, 'total_flos': 9798451200000.0, 'train_loss': 0.7338128089904785, 'epoch': 3.0})

In [51]:
# Save the fine-tuned model
trainer.save_model("./fine_tuned_chatbot")
tokenizer.save_pretrained("./fine_tuned_chatbot")

('./fine_tuned_chatbot/tokenizer_config.json',
 './fine_tuned_chatbot/special_tokens_map.json',
 './fine_tuned_chatbot/vocab.json',
 './fine_tuned_chatbot/merges.txt',
 './fine_tuned_chatbot/added_tokens.json',
 './fine_tuned_chatbot/tokenizer.json')

In [52]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the fine-tuned model
model = AutoModelForCausalLM.from_pretrained("./fine_tuned_chatbot")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_chatbot")

# Ensure the pad token is set properly
tokenizer.pad_token = tokenizer.eos_token

def chat_with_model(prompt):
    # Prepare input text
    input_text = f"<|startoftext|>{prompt}"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=128)

    # Generate response with attention mask
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=150,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=1,
        # temperature=0.7,  # Adjust for randomness in output
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Return the generated response
    return response.split(prompt)[-1].strip()



In [54]:
# Example usage
print(chat_with_model("How can I access the e-library?"))

You can access the e-library through the student portal under the 'Access to the e-library' section.
